In [25]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_community.document_loaders import PyPDFLoader
from langchain import hub
from langchain_core.output_parsers import StrOutputParser

In [17]:
load_dotenv()
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["LANGSMITH_ENDPOINT"] = os.getenv("LANGSMITH_ENDPOINT")
os.environ["LANGSMITH_PROJECT"] = os.getenv("LANGSMITH_PROJECT")
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["SERPAPI_API_KEY"]=os.getenv("SERPAPI_API_KEY")

In [18]:
resume = PyPDFLoader("data/resume_sanjay.pdf")
documents=resume.load()
resume_text = "\n".join([doc.page_content for doc in documents])
resume_text

'Sanjay Praveen \n \nEmail: pappalasanjay.prave@hcl.com Mobile: 7893464305 \n \n \n \nProfessional Summary \n \n• Aspiring for a challenging career in Software Development as to use my learned skills and \nexperience for best results. \n• Having 6 years 4 months of experience in Software Development with J2EE,Rpa Blue prism \ndeveloper and RPA Business Analyst \n• Exposure in end-to-end development of software products, right from requirement analysis, \nsystem study, designing, coding, testing, documentation and implementation using  diverse \ntechnologies. \n• Strong analytical, problem solving & organizational building abilities. \n• Worked in agile and waterfall methodologies with high quality deliverables and delivered  on-time. \n• Good knowledge of various Java/J2ee Architectures, Blue Prism Frameworks and their \nimplementation. \n• Proficient at grasping new technical concepts quickly and utilizing the same in a productive \nmanner. \n• Well-equipped with Object Oriented Conce

In [29]:
prompt = hub.pull("aicraft/resume_details_extractor")
prompt

d:\personal_repo\envs\agenv\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


ChatPromptTemplate(input_variables=['resume_text'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'aicraft', 'lc_hub_repo': 'resume_details_extractor', 'lc_hub_commit_hash': 'eea86b30a7fa69f0afd429cdfa05972ea059818fe8e1684b014f03764d0cc31a'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['resume_text'], input_types={}, partial_variables={}, template='Extract and respond ONLY with a valid JSON object. Do not add any explanation, markdown, or comments.\n\nReturn fields: experience, summary, skills (categorized), projects, education.\n\nResume:\n\\"\\"\\"\n{resume_text}\n\\"\\"\\"'), additional_kwargs={})])

In [21]:
llm = ChatGroq(temperature=0.7, model="llama-3.3-70b-versatile")

In [26]:
stropparser=StrOutputParser()

In [27]:
chain = prompt | llm | stropparser

In [30]:
out=chain.invoke({"resume_text": resume_text})
out

'Here\'s a JSON object with the requested fields:\n\n```json\n{\n  "Experience": [\n    {\n      "company": "Tata Consultancy Services",\n      "duration": "2016-2019",\n      "role": "Developer",\n      "projects": [\n        "NPD (Non Provisional Data System)",\n        "Data Adjustment utility",\n        "Remediate to data quality (R2DQ)",\n        "Kaiser Clinical Library",\n        "Kaiser KP VINE",\n        "Kaiser KP Phonebook",\n        "Kaiser Resource program management"\n      ]\n    },\n    {\n      "company": "HCL Technologies",\n      "duration": "2019-till date",\n      "role": "RPA Developer/Lead",\n      "projects": [\n        "CA PPM",\n        "KP Phonebook",\n        "Membership",\n        "Claims Track",\n        "Claims",\n        "Nestle"\n      ]\n    }\n  ],\n  "Summary": "Aspiring for a challenging career in Software Development with 6 years 4 months of experience in Software Development with J2EE, RPA Blue prism developer and RPA Business Analyst. Strong anal

In [31]:
def clean_json_block(raw_response: str) -> str:
    start = raw_response.find('{')
    end = raw_response.rfind('}') + 1
    return raw_response[start:end]

In [32]:
final = clean_json_block(out)
final

'{\n  "Experience": [\n    {\n      "company": "Tata Consultancy Services",\n      "duration": "2016-2019",\n      "role": "Developer",\n      "projects": [\n        "NPD (Non Provisional Data System)",\n        "Data Adjustment utility",\n        "Remediate to data quality (R2DQ)",\n        "Kaiser Clinical Library",\n        "Kaiser KP VINE",\n        "Kaiser KP Phonebook",\n        "Kaiser Resource program management"\n      ]\n    },\n    {\n      "company": "HCL Technologies",\n      "duration": "2019-till date",\n      "role": "RPA Developer/Lead",\n      "projects": [\n        "CA PPM",\n        "KP Phonebook",\n        "Membership",\n        "Claims Track",\n        "Claims",\n        "Nestle"\n      ]\n    }\n  ],\n  "Summary": "Aspiring for a challenging career in Software Development with 6 years 4 months of experience in Software Development with J2EE, RPA Blue prism developer and RPA Business Analyst. Strong analytical, problem-solving, and organizational building abilitie